In [113]:
import requests
from requests_oauthlib import OAuth1
from bs4 import BeautifulSoup
DATA_BASEURL = 'https://babel.hathitrust.org/cgi/htd/volume/'
#SECURE_DATA_BASEURL = 
auth_key = '998e5140ec'
auth_secret = 'cbc998dee1350bddf5af0bcd09cb'

In [114]:
api = DataAPI(auth_key, auth_secret)

In [137]:
#Donne's (I think) Illiad/Odyssey translation.
print(api.getpageocr('nyp.33433082228226', 136).decode("utf-8"))

https://babel.hathitrust.org/cgi/htd/volume/pageocr/nyp.33433082228226/136
426
Book XIII.
THE ODYSSEY..
'Can mighty Neptune thus of man complain?
Neptune, tremendous o'er the boundless
main!
Revered and awful even in heaven's abodes,
Ancient and great ! a god above the gods!
If that low race offend thy power divine,
(Weak, daring creatures !) is not vengeance
thine?
Go, then, the guilty at thy will chastise.'
He said: the shaker of the earth replies:
'This then I doom ; to fix the gallant ship
A mark of vengeance on the sable deep:
To warn the thoughtless self-confiding train,
No more unlicensed thus to brave the main.
Full in their port a shady hill shall rise,
If such thy will.'—' We will it, (Jove replies)
Even when with transport blackening all the
strand,
The swarming people hail their ship to land,
Fix her for ever, a memorial stone:
Still let her seem to sail, and seem alone;
The trembling crowds shall see the sudden
shade
Of whelming mountainsoverhang their head!'
With that, th

In [108]:
#This is some code I grabbed off github. I need to finish updating it. only a few things work rn.s

class DataAPI(object):

    def __init__(self, client_key, client_secret, secure=False):
        """ Initialize a DataAPI object.
        Args:
            client_key: OAuth client key (registered with HathiTrust)
            client_secret: secret OAuth key
            secure: toggles http/https session. Defaults to
                 http, use https for access to restricted content.
        Initializes a persistent Requests session and attaches 
        OAuth credentials to the session. All queries are performed as 
        method calls on the HTDataInterface object.
        For now, all queries return the raw content string, rather than
        processing the json or xml structures.
        """

        self.client_key = client_key
        self.client_secret = client_secret
        self.oauth = OAuth1(client_key=client_key, 
                            client_secret=client_secret, 
                            signature_type='query')

        self.rsession = requests.Session()
        self.rsession.auth = self.oauth

        if secure:
            self.baseurl = SECURE_DATA_BASEURL
        else: 
            self.baseurl = DATA_BASEURL


    def _makerequest(self, resource, doc_id, sequence=None, 
                        v=2, json=False, callback=None):
        """ Construct and perform URI request.
        Args:
            resource: resource type
            doc_id: document identifier of target
            sequence: page number for single page resources
            v: API version 
            json: if json=True, the json representation of
                the resource is returned. Only valid for resources that 
                are xml or xml+atom by default.
            callback: optional javascript callback function, 
                which only has an effect if json=True.
        Return: 
            content of the response, in bytes
        Note there's not much error checking on url construction, 
        but errors do get raised after badly formed requests. 
        To do: implement some exception checking here, and identify 
        what sort of errors are being returned (eg. BadRequest, 
        Unauthorized, NotFound, etc.)   
        """

        url = "".join([self.baseurl, resource, '/', doc_id])
        
        if sequence:
            url += '/' + str(sequence)
        print(url)
        params = {'v': str(v)}
        if json:
            params['format'] = 'json'
            if callback:
                params['callback'] = callback
        
        #url='https://babel.hathitrust.org/cgi/htd/volume/pageocr/nyp.33433082228226/302'
        r = self.rsession.get(url, params=params)
        r.raise_for_status()

        return r.content


    def getmeta(self, doc_id, json=False):
        """ Retrieve Volume and Rights Metadata resources.
        Args:
            doc_id: document identifier
            json: if json=True, the json representation of
                the resource is returned, otherwise efaults to an atom+xml 
                format.
        Return: 
            xml or json string
        """
        return self._makerequest('meta', doc_id, json=json)


    def getstructure(self, doc_id, json=False):
        """ Retrieve a METS document.
        Args:
            doc_id: target document
            json: toggles json/xml 
        Return:
            xml or json string
        """
        return self._makerequest('structure', doc_id, json=json)


    def getpagemeta(self, doc_id, seq, json=False):
        """ Retrieve single page metadata. """
        return self._makerequest('pagemeta', doc_id, sequence=seq, json=json)


    def getaggregate(self, doc_id):
        """ Get aggregate record data. 
        Return: 
            zip content that contains tiff/jp2/jpeg, .txt OCR files,
                + Source METS (not the same as Hathi METS)
        """
        return self._makerequest('aggregate', doc_id)


    def getpageimage(self, doc_id, seq):
        """ Retrieve Single Page Image.
        Return:
            response with tiff, jp2, or jpeg file
        """
        return self._makerequest('pageimage', doc_id, sequence=seq)


    def getpageocr(self, doc_id, sequence):
        """  Get single-page OCR.
        Return:
            UTF-8 encoded OCR plain text
        """
        return self._makerequest('pageocr', doc_id, sequence=sequence)


    def getpagecoordocr(self, doc_id, sequence):
        """ Get single-page coordinate OCR.
        Return:
            UTF-8 encoded XML OCR
        """
        return self._makerequest('pagecoordocr', doc_id, sequence=sequence)

In [135]:
[1,2,3]+[4,5]

[1, 2, 3, 4, 5]